In [1]:
import os
import time
import pickle
from collections import deque

import gym
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layers

from matplotlib import pyplot as plt
from IPython.display import clear_output

RANDOM_SEED = 123
STATE_SIZE = 8
ACTION_SIZE = 4
BUFFER_SIZE = 10


In [5]:
def get_history(path, mov=10):
    with open(path, 'rb') as f:
        history = np.array(pickle.load(f))
        elapseds = history[:,0]
        episodes = history[:,1]
        scores = history[:,2]
        steps = history[:,3]
        mov_avg_scores = np.array([scores[max(i-mov+1, 0):i+1].mean() for i in range(len(scores))])
        mov_avg_steps = np.array([steps[max(i-mov+1, 0):i+1].mean() for i in range(len(steps))])
        acc_scores = np.array([scores[0:i+1].sum() for i in range(len(scores))])
        acc_steps = np.array([steps[0:i+1].sum() for i in range(len(steps))])
    return scores, elapseds, episodes, mov_avg_scores, mov_avg_steps, acc_scores, elapseds, acc_steps

history = get_history('history/b15-s100.pkl')
history[0][:10]

array([-258.2058322 , -136.86985035, -111.59668666, -220.82742444,
       -220.8832746 , -138.91868609, -210.99246184, -256.89713443,
       -332.20396578,  -31.83366717])

In [2]:
def get_history(path, mov=10):
    with open(path, 'rb') as f:
        history = np.array(pickle.load(f))
        elapseds = history[:,0]
        episodes = history[:,1]
        scores = history[:,2]
        steps = history[:,3]
        mov_avg_scores = np.array([scores[max(i-mov+1, 0):i+1].mean() for i in range(len(scores))])
        mov_avg_steps = np.array([steps[max(i-mov+1, 0):i+1].mean() for i in range(len(steps))])
        acc_scores = np.array([scores[0:i+1].sum() for i in range(len(scores))])
        acc_steps = np.array([steps[0:i+1].sum() for i in range(len(steps))])
    return elapseds, episodes, mov_avg_scores, mov_avg_steps, acc_scores, elapseds, acc_steps

MOV = 100
history_b1_s0 = get_history(os.path.join('./a2c_b1_s0', 'history.pkl'), mov=MOV)
history_b5_s0 = get_history(os.path.join('./a2c_b5_s0', 'history.pkl'), mov=MOV)
history_b10_s0 = get_history(os.path.join('./a2c_b10_s0', 'history.pkl'), mov=MOV)
history_b15_s0 = get_history(os.path.join('./a2c_b15_s0', 'history.pkl'), mov=MOV)
history_b20_s0 = get_history(os.path.join('./a2c_b20_s0', 'history.pkl'), mov=MOV)
history_b30_s0 = get_history(os.path.join('./a2c_b30_s0', 'history.pkl'), mov=MOV)
history_b50_s0 = get_history(os.path.join('./a2c_b50_s0', 'history.pkl'), mov=MOV)

history_b1_s1 = get_history(os.path.join('./a2c_b1_s1', 'history.pkl'), mov=MOV)
history_b5_s1 = get_history(os.path.join('./a2c_b5_s1', 'history.pkl'), mov=MOV)
history_b10_s1 = get_history(os.path.join('./a2c_b10_s1', 'history.pkl'), mov=MOV)
history_b15_s1 = get_history(os.path.join('./a2c_b15_s1', 'history.pkl'), mov=MOV)
history_b20_s1 = get_history(os.path.join('./a2c_b20_s1', 'history.pkl'), mov=MOV)

history_b10_s3 = get_history(os.path.join('./a2c_b10_s3', 'history.pkl'), mov=MOV)
history_b10_s5 = get_history(os.path.join('./a2c_b10_s5', 'history.pkl'), mov=MOV)
history_b10_s10 = get_history(os.path.join('./a2c_b10_s10', 'history.pkl'), mov=MOV)
history_b10_s15 = get_history(os.path.join('./a2c_b10_s15', 'history.pkl'), mov=MOV)
history_b10_s20 = get_history(os.path.join('./a2c_b10_s20', 'history.pkl'), mov=MOV)
history_b10_s30 = get_history(os.path.join('./a2c_b10_s30', 'history.pkl'), mov=MOV)
history_b10_s50 = get_history(os.path.join('./a2c_b10_s50', 'history.pkl'), mov=MOV)
history_b10_s70 = get_history(os.path.join('./a2c_b10_s70', 'history.pkl'), mov=MOV)
history_b10_s100 = get_history(os.path.join('./a2c_b10_s100', 'history.pkl'), mov=MOV)

history_b15_s3 = get_history(os.path.join('./a2c_b15_s3', 'history.pkl'), mov=MOV)
history_b15_s5 = get_history(os.path.join('./a2c_b15_s5', 'history.pkl'), mov=MOV)
history_b15_s10 = get_history(os.path.join('./a2c_b15_s10', 'history.pkl'), mov=MOV)
history_b15_s15 = get_history(os.path.join('./a2c_b15_s15', 'history.pkl'), mov=MOV)
history_b15_s20 = get_history(os.path.join('./a2c_b15_s20', 'history.pkl'), mov=MOV)
history_b15_s30 = get_history(os.path.join('./a2c_b15_s30', 'history.pkl'), mov=MOV)
history_b15_s50 = get_history(os.path.join('./a2c_b15_s50', 'history.pkl'), mov=MOV)
history_b15_s70 = get_history(os.path.join('./a2c_b15_s70', 'history.pkl'), mov=MOV)
history_b15_s100 = get_history(os.path.join('./a2c_b15_s100', 'history.pkl'), mov=MOV)

In [8]:
BUFFER_SIZE = 10
STEP_SIZE = 100
EPISODE = 3000

SAVED_DATA_DIR = f'model'
MODEL_NALE = f'b10-s100'

env = gym.make('LunarLander-v2')
env.viewer = None


class ActorModel(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(__class__, self).__init__()
        self.dense1 = layers.Dense(512, activation='relu')
        self.out = layers.Dense(action_size, activation='softmax')
        self.build(input_shape=(None, state_size))
    
    def call(self, x):
        x = self.dense1(x)
        policy = self.out(x)
        return policy

model = ActorModel(BUFFER_SIZE * STATE_SIZE, ACTION_SIZE)
model.load_weights(os.path.join(SAVED_DATA_DIR, MODEL_NALE))

for episode in range(1, 8):
    score = 0
    step = 0
    buffer = deque(np.zeros((BUFFER_SIZE, STATE_SIZE), dtype='float32'), maxlen=BUFFER_SIZE)
    
    done = False
    state = env.reset()
    buffer.append(state)
    state = np.array(buffer).flatten()
    
    while not done:
        env.render()
        policy = np.array(model(state.reshape(1, -1))[0])
        action = policy.argmax()
#         action = np.random.choice(ACTION_SIZE, 1, p=policy)[0]
        next_state, reward, done, _ = env.step(action)
        buffer.append(next_state)
        next_state = np.array(buffer).flatten()
        score += reward
        step += 1
        state = next_state
        
    print("episode: {:4d} | score: {:4.2f} | step: {:4d}".format(episode, score, step))
env.close()

episode:    1 | score: 231.25 | step:  278
episode:    2 | score: 289.34 | step:  316
episode:    3 | score: 251.82 | step:  360
episode:    4 | score: 230.84 | step:  331
episode:    5 | score: 240.23 | step:  317
episode:    6 | score: 250.93 | step:  315
episode:    7 | score: 269.46 | step:  383
